In [7]:
# from helper_functions.tourney_sim import simulate_tournament
import os
import numpy as np
import pickle
from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_columns', 90)

In [8]:
all_lists = []
for f in os.listdir('brackets'):
        with open('brackets/{}'.format(f), 'rb') as file:
            data=pickle.load(file)
        all_lists += list(data.values())

In [9]:
len(all_lists)

7779

In [10]:
h_changes = 0
v_changes=0
b_changes=0
for i in tqdm(range(len(all_lists))):
    for j in range(len(all_lists[i])):
        if all_lists[i][j] == 'Howard':
            all_lists[i][j] = 'Wagner'
            h_changes+=1
        if all_lists[i][j] == 'Virginia':
            all_lists[i][j] = 'Colorado St.'
            v_changes += 1
        if all_lists[i][j] == 'Boise St.':
            all_lists[i][j] = 'Colorado'
            b_changes += 1


100%|██████████| 7779/7779 [00:00<00:00, 112744.96it/s]


In [11]:
brackets_series = pd.Series(all_lists)

In [12]:
df=pd.DataFrame(all_lists)


In [13]:
real_winners = ['Michigan St.','Duquesne','Creighton', 'Arizona', 'North Carolina','Illinois','Oregon', 'Dayton',
                'Texas','Oakland','Gonzaga','Iowa St.','Tennessee', 'NC State', 'Washington St.','Kansas',
                'Northwestern','Baylor','San Diego St.','Marquette','Clemson','UConn','Yale','Colorado',
                'Texas A&M','Duke','Purdue','Alabama','James Madison', 'Houston','Utah St.','Grand Canyon']
real_losers = ['Mississippi St.','BYU', 'Akron', 'Long Beach St.', 'Wagner', 'Morehead St.','South Carolina','Nevada',
               'Colorado St.', 'Kentucky','McNeese','South Dakota St.',"Saint Peter's", 'Texas Tech','Drake','Samford',
               'Fla. Atlantic','Colgate','UAB','Western Ky.','New Mexico','Stetson','Auburn','Florida',
               'Nebraska',"Vermont",'Grambling','Col. of Charleston','Wisconsin','Longwood',"TCU","Saint Mary's (CA)"]

In [14]:
game_ids = [9, 4, 29, 15, 8, 5, 28, 14, 30, 21, 26, 7, 31, 20,6,27,
            1,13,2,23,12,0,3,22,17,19,24,11,18,16,25,10]

In [15]:
# df.head()

In [16]:
outcomes = dict(zip(game_ids, real_winners))
losses=dict(zip(game_ids, real_losers))

In [22]:
for item in outcomes.keys():
    print(df[df[item]==outcomes[item]].shape)

(3995, 63)
(1987, 63)
(6441, 63)
(6407, 63)
(6805, 63)
(4911, 63)
(3138, 63)
(5429, 63)
(5803, 63)
(2311, 63)
(4753, 63)
(6553, 63)
(5667, 63)
(1732, 63)
(5189, 63)
(4815, 63)
(1462, 63)
(5485, 63)
(6552, 63)
(6525, 63)
(6010, 63)
(6594, 63)
(1532, 63)
(4762, 63)
(3315, 63)
(5446, 63)
(7115, 63)
(3927, 63)
(3980, 63)
(4330, 63)
(5696, 63)
(2366, 63)


In [17]:
#scoring - 10 points first round, 20 second round, 40 third round, etc.
def current_score(dataframe, winners_dict):
    tot=0
    for i in winners_dict.keys():
        if i < 32:
            tot += 10*(dataframe[i] == winners_dict[i])
    return tot

def max_remaining(dataframe, losers):
    possible_scores = []
    for j in range(len(dataframe)):
        poss=1920
        first_round = dataframe.iloc[j][:32]
        second_round = dataframe.iloc[j][32:48]
        third_round = dataframe.iloc[j][48:56]
        fourth_round = dataframe.iloc[j][56:60]
        fifth_round = dataframe.iloc[j][60:62]
        finals = dataframe.iloc[j][62]
        for i in losers:
            if i in finals:
                poss -= 320
            if i in list(fifth_round):
                poss -= 160
            if i in list(fourth_round):
                poss -= 80
            if i in list(third_round):
                poss -= 40
            if i in list(second_round):
                poss -= 20
            if i in list(first_round):
                poss -= 10
        possible_scores.append(poss)
    return possible_scores

In [18]:
dfc=df.copy()

In [19]:
dfc['Score'] = current_score(df, outcomes)

In [20]:
dfc['Max']=max_remaining(dfc, real_losers)

In [21]:
dfc.sort_values(by='Max', ascending=False)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,Score,Max
6938,UConn,Northwestern,San Diego St.,Auburn,Duquesne,Illinois,Washington St.,Iowa St.,North Carolina,Mississippi St.,Saint Mary's (CA),Alabama,Clemson,Baylor,Dayton,Arizona,Houston,Nebraska,James Madison,Duke,NC State,Oakland,Colorado,Western Ky.,Purdue,Utah St.,Gonzaga,Kansas,Oregon,Creighton,Colorado St.,Tennessee,Northwestern,San Diego St.,Illinois,Iowa St.,North Carolina,Alabama,Clemson,Dayton,Houston,James Madison,NC State,Colorado,Utah St.,Gonzaga,Creighton,Tennessee,San Diego St.,Illinois,North Carolina,Dayton,James Madison,NC State,Gonzaga,Creighton,San Diego St.,North Carolina,James Madison,Creighton,San Diego St.,Creighton,San Diego St.,260,1860
2848,UConn,Fla. Atlantic,San Diego St.,Auburn,Duquesne,Illinois,Drake,Iowa St.,North Carolina,Michigan St.,Saint Mary's (CA),Alabama,Clemson,Baylor,Dayton,Arizona,Houston,Texas A&M,James Madison,Duke,NC State,Kentucky,Colorado,Marquette,Purdue,Utah St.,Gonzaga,Kansas,Oregon,Creighton,Texas,Tennessee,UConn,San Diego St.,Duquesne,Iowa St.,North Carolina,Saint Mary's (CA),Baylor,Dayton,Texas A&M,James Madison,NC State,Marquette,Purdue,Gonzaga,Oregon,Tennessee,UConn,Duquesne,North Carolina,Baylor,James Madison,Marquette,Purdue,Oregon,Duquesne,Baylor,James Madison,Purdue,Duquesne,James Madison,James Madison,270,1850
4075,UConn,Fla. Atlantic,San Diego St.,Auburn,Duquesne,Illinois,Washington St.,Iowa St.,North Carolina,Michigan St.,Grand Canyon,Alabama,Clemson,Baylor,Dayton,Arizona,Longwood,Texas A&M,Wisconsin,Duke,NC State,Oakland,Florida,Marquette,Purdue,TCU,Gonzaga,Samford,Oregon,Creighton,Texas,Tennessee,UConn,San Diego St.,Duquesne,Iowa St.,North Carolina,Grand Canyon,Clemson,Dayton,Texas A&M,Duke,NC State,Marquette,Purdue,Gonzaga,Creighton,Tennessee,UConn,Duquesne,North Carolina,Clemson,Duke,Marquette,Gonzaga,Tennessee,UConn,Clemson,Duke,Gonzaga,UConn,Duke,UConn,250,1850
2278,UConn,Northwestern,San Diego St.,Auburn,BYU,Illinois,Drake,Iowa St.,North Carolina,Michigan St.,Saint Mary's (CA),Col. of Charleston,Clemson,Baylor,Dayton,Arizona,Houston,Texas A&M,James Madison,Duke,NC State,Oakland,Florida,Marquette,Purdue,Utah St.,Gonzaga,Kansas,Oregon,Creighton,Texas,Tennessee,UConn,San Diego St.,Illinois,Iowa St.,North Carolina,Saint Mary's (CA),Baylor,Dayton,Houston,James Madison,NC State,Marquette,Purdue,Gonzaga,Creighton,Tennessee,UConn,Illinois,North Carolina,Dayton,Houston,NC State,Gonzaga,Creighton,UConn,North Carolina,Houston,Gonzaga,UConn,Gonzaga,UConn,260,1840
844,UConn,Fla. Atlantic,San Diego St.,Yale,BYU,Illinois,Washington St.,Iowa St.,North Carolina,Mississippi St.,Grand Canyon,Col. of Charleston,Clemson,Colgate,Dayton,Arizona,Houston,Texas A&M,Wisconsin,Duke,NC State,Kentucky,Colorado,Marquette,Purdue,Utah St.,Gonzaga,Kansas,Oregon,Akron,Texas,Tennessee,UConn,San Diego St.,Illinois,Iowa St.,North Carolina,Grand Canyon,Clemson,Dayton,Houston,Duke,NC State,Marquette,Purdue,Gonzaga,Oregon,Texas,UConn,Illinois,North Carolina,Dayton,Duke,NC State,Purdue,Texas,Illinois,Dayton,Duke,Texas,Illinois,Duke,Illinois,240,1840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5991,UConn,Fla. Atlantic,San Diego St.,Yale,BYU,Morehead St.,Washington St.,Iowa St.,North Carolina,Mississippi St.,Saint Mary's (CA),Alabama,New Mexico,Colgate,Dayton,Long Beach St.,Houston,Texas A&M,Wisconsin,Vermont,NC State,Oakland,Florida,Marquette,Purdue,TCU,McNeese,Samford,South Carolina,Creighton,Texas,Tennessee,Fla. Atlantic,Yale,Morehead St.,Washington St.,Mississippi St.,Saint Mary's (CA),New Mexico,Long Beach St.,Houston,Wisconsin,NC State,Florida,TCU,McNeese,South Carolina,Tennessee,Fla. Atlantic,Morehe

In [24]:
dfc[dfc['Max']==dfc['Max'].max()]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,Score,Max
4075,UConn,Fla. Atlantic,San Diego St.,Auburn,Duquesne,Illinois,Washington St.,Iowa St.,North Carolina,Michigan St.,Grand Canyon,Alabama,Clemson,Baylor,Dayton,Arizona,Longwood,Texas A&M,Wisconsin,Duke,NC State,Oakland,Florida,Marquette,Purdue,TCU,Gonzaga,Samford,Oregon,Creighton,Texas,Tennessee,UConn,San Diego St.,Duquesne,Iowa St.,North Carolina,Grand Canyon,Clemson,Dayton,Texas A&M,Duke,NC State,Marquette,Purdue,Gonzaga,Creighton,Tennessee,UConn,Duquesne,North Carolina,Clemson,Duke,Marquette,Gonzaga,Tennessee,UConn,Clemson,Duke,Gonzaga,UConn,Duke,UConn,150,1910


In [108]:
for c in df.columns:
    vc=df[c].value_counts()
    print('{}: {}%'.format(vc.index[0],np.round((vc.iloc[0]/len(df))*100,2)))

UConn: 84.77%
Fla. Atlantic: 81.21%
San Diego St.: 84.23%
Auburn: 80.31%
BYU: 74.46%
Illinois: 63.13%
Washington St.: 66.71%
Iowa St.: 84.24%
North Carolina: 87.48%
Michigan St.: 51.36%
Saint Mary's (CA): 69.58%
Alabama: 50.48%
Clemson: 77.26%
Baylor: 70.51%
Dayton: 69.79%
Arizona: 82.36%
Houston: 55.66%
Nebraska: 57.39%
James Madison: 51.16%
Duke: 70.01%
Texas Tech: 77.73%
Kentucky: 70.29%
Colorado: 61.22%
Marquette: 83.88%
Purdue: 91.46%
Utah St.: 73.22%
Gonzaga: 61.1%
Kansas: 61.9%
South Carolina: 59.66%
Creighton: 82.8%
Texas: 74.6%
Tennessee: 72.85%
UConn: 69.55%
San Diego St.: 55.39%
BYU: 55.08%
Washington St.: 39.4%
North Carolina: 64.53%
Saint Mary's (CA): 54.76%
Clemson: 50.04%
Dayton: 50.75%
Houston: 35.74%
Duke: 32.43%
Texas Tech: 54.44%
Colorado: 42.24%
Purdue: 66.37%
Gonzaga: 48.01%
Creighton: 43.12%
Texas: 50.38%
UConn: 55.84%
BYU: 41.53%
North Carolina: 40.37%
Clemson: 28.08%
Duke: 18.27%
Texas Tech: 36.3%
Purdue: 41.62%
Creighton: 38.21%
UConn: 46.47%
North Carolina: 28